In [4]:
# Packages used in this project implementation
import random
import os
import time
import itertools
import h5py
import math
import csv
import tenseal as ts
import numpy as np
import pandas as pd
from imutils import paths
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential, model_from_json
from tensorflow.keras.layers import Dense
from tensorflow_privacy.privacy.optimizers.dp_optimizer_keras import DPKerasSGDOptimizer


Create Client Dataset 

In [5]:
def create_clients(data_list, num_clients=10, initial='clients'):
    ''' return: a dictionary with keys clients' names and value as 
                data shards - tuple of images and label lists.
        args: 
            image_list: a list of numpy arrays of training images
            label_list:a list of binarized labels for each image
            num_client: number of fedrated members (clients)
            initials: the clients'name prefix, e.g, clients_1 
            
    '''

    #create a list of client names
    client_names = ['{}_{}'.format(initial, i+1) for i in range(num_clients)]

    #randomize the data
    data = list(data_list)

    #shard data and place at each client
    size = len(data)//num_clients
    shards = [data[i:i + size] for i in range(0, size*num_clients, size)]

    #number of clients must equal number of shards
    assert(len(shards) == len(client_names))

    return {client_names[i] : shards[i] for i in range(len(client_names))} 

Create Dataset through sliding window

In [6]:
def windowed_dataset(client_data, window_size, batch_size, shuffle_buffer):
  dataset = tf.data.Dataset.from_tensor_slices(client_data)
  dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True)
  dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))
  dataset = dataset.shuffle(shuffle_buffer).map(lambda window: (window[:-1], window[-1]))
  dataset = dataset.batch(batch_size).prefetch(1)
  return dataset

Class for Model training, aggregators methods

In [8]:
class SimpleMLP:
    @staticmethod
    def build():
        tf.keras.backend.clear_session()
        tf.random.set_seed(51)
        np.random.seed(51)
        model = tf.keras.models.Sequential([ tf.keras.layers.Lambda(lambda x: tf.expand_dims(x, axis=-1), 
                                                                    input_shape=[None]), tf.keras.layers.SimpleRNN(400, return_sequences=True), 
                                            tf.keras.layers.SimpleRNN(400), tf.keras.layers.Dense(1), ])
        return model

def weight_scalling_factor(clients_trn_data, client_name):
    client_names = list(clients_trn_data.keys())
    global_count = sum([len(clients_trn_data[client_nam]) for client_nam in client_names])
    local_count = len(clients_trn_data[client_name])
    return local_count/global_count


def scale_model_weights(weight, scalar):
    '''function for scaling a models weights'''
    weight_final = []
    steps = len(weight)
    for i in range(steps):
        weight_final.append(scalar * weight[i])
    return weight_final



def sum_scaled_weights(scaled_weight_list):
    '''Return the sum of the listed scaled weights. The is equivalent to scaled avg of the weights'''
    avg_grad = list()
    #get the average grad accross all client gradients
    for grad_list_tuple in zip(*scaled_weight_list):
        layer_mean = tf.math.reduce_sum(grad_list_tuple, axis=0)
        avg_grad.append(layer_mean)
        
    return avg_grad


def test_model(data, model, window_size):
    #forecast (predictions) for every point in the time series
    series_trans = np.array(data)
    forecast = []
    for time in range(min(100,len(data)-window_size)):
        forecast.append(model.predict(series_trans[time:time + window_size][np.newaxis]))
    forecast = forecast[:]
    results = np.array(forecast)[:, 0, 0]
    print("MSE on the validation dataset: ", tf.keras.metrics.mean_squared_error(series_trans[window_size:min(100+window_size,len(data))], results).numpy())
    print("MAE on the validation dataset: ", tf.keras.metrics.mean_absolute_error(series_trans[window_size:min(100+window_size,len(data))], results).numpy())
    return series_trans[window_size:min(100+window_size,len(data))], results

def mean_weights(weight_list):
    '''Return the mean of the weights'''
    avg_grad = list()
    for grad_list_tuple in zip(*weight_list):
        layer_mean = np.mean(grad_list_tuple, axis=0)
        avg_grad.append(layer_mean)
        
    return avg_grad

def median_weights(weight_list):
    '''Return the median of weights. '''
    avg_grad = list()
    for grad_list_tuple in zip(*weight_list):
        layer_mean = np.median(grad_list_tuple, axis=0)
        avg_grad.append(layer_mean)
        
    return avg_grad


def trim_weights(weight_list, a):
    '''Return the mean of of weights without the highest a and lowest a values. '''
    avg_grad = list()
    for grad_list_tuple in zip(*weight_list):
        layer_mean = np.mean(np.sort(grad_list_tuple)[a:-a], axis=0)
        avg_grad.append(layer_mean)
        
    return avg_grad

def encrypt_weights(model, a, context):
    l = scale_model_weights(model.get_weights(),a)
    enc_weights = []
    for j in range(len(l)):
        if len(l[j].shape)==1:
            enc_weights.append(ts.ckks_vector(ctx_eval, l[j]))
        else:
            t = []
            for i in range(len(l[j])):
                t.append(ts.ckks_vector(ctx_eval, l[j][i]))
            enc_weights.append(t)
    return enc_weights


def decrypt_weights(enc_weights, context):
    s = []
    for j in range(len(enc_weights)):
        if type(enc_weights[j]) != list:
            s.append(np.array((enc_weights[j].decrypt())))
        else:
            a = []
            for i in range(len(enc_weights[j])):
                a.append(np.array((enc_weights[j][i]).decrypt()))
            s.append(np.array(a))
    return s


def add_noise(weights, context):
    s = []
    l = []
    for j in range(len(weights)):
        if type(weights[j]) != list:
            a = random.uniform(0, 1)
            s.append(((weights[j])+a))
            l.append((a))
        else:
            a = random.uniform(0, 1)
            t1 = []
            t2 = []
            for i in range(len(weights[j])):
                t1.append(((weights[j][i])+a))
                t2.append((a))
            s.append((t1))
            l.append((t2))
    return s, l


def remove_noise(weights, N, context):
    s = []
    for j in range(len(weights)):
        if type(weights[j]) != list:
            A = 0
            for l in range(len(N)):
                A = A + N[l][j]
            s.append(((weights[j])-A))
        else:
            t1 = []
            for i in range(len(weights[j])):
                A = 0
                for l in range(len(N)):
                    A = A + N[l][j][i]
                t1.append(((weights[j][i])-A))
            s.append((t1))
    return s


def sum_enc_weights(L):
    s = []
    for j in range(len(L[0])):
        if type(L[0][j]) != list:
            z = L[0][j]
            for i in range(1,len(L)):
                z = z+ L[i][j]
            s.append(z)
        else:
            a = []
            for i in range(len(L[0][j])):
                z = L[0][j][i]
                for k in range(1,len(L)):
                    z = z+ L[k][j][i]
                a.append(z)
            s.append(a)
    return s

def Training(dataset, epochs =100):
    dataset = np.array(dataset)
    window_size = 20
    batch_size = 16
    modl = SimpleMLP()
    model = modl.build()
    model.compile(loss="mse", optimizer=tf.keras.optimizers.SGD(learning_rate=3e-4, momentum=0.9), metrics=["mae"])
    datasets = windowed_dataset(dataset, window_size, batch_size, len(dataset))
    model.fit(datasets, epochs, verbose=0)
    return model

def Plots(I, x,y):
    X = I

    # Plotting both the curves simultaneously
    plt.plot(X, y, color='r', label='Forecasted values')
    plt.plot(X, x, color='g', label='Original values')

    plt.xlabel("Time steps")
    plt.ylabel("Traffic volume")

    plt.legend()

    plt.show()

def FedProx(local_model, global_model, mu:int, optimizer, dataset, loss_fn):
    """Train `model` on one epoch of `train_data`"""
#     optimizer = tf.keras.optimizers.SGD(learning_rate=3e-4)
#     loss_fn = tf.keras.losses.MeanSquaredError()
    for step, (x,y) in enumerate(dataset):
        with tf.GradientTape() as tape:
            y_hat = local_model(x)
            loss = loss_fn(y, y_hat)
            distance = squared_distance(weight_to_vector(local_model.get_weights()), weight_to_vector(global_model.get_weights()))
            loss+=mu/2*distance
            gradients = tape.gradient(loss, local_model.trainable_weights)
        optimizer.apply_gradients(zip(gradients,local_model.trainable_weights))
    return local_model

In [ ]:
def create_test_data_sets(dataset, num_datasets=8, initial='test'):

    #create a list of client names
    test_names = ['{}_{}'.format(initial, i+1) for i in range(num_datasets)]

    #randomize the data
    data = list(dataset)

    #shard data and place at each client
    size = len(data)//num_datasets
    shards = [data[i:i + size] for i in range(0, size*num_datasets, size)]

    #number of clients must equal number of shards
    assert(len(shards) == len(test_names))

    return {test_names[i] : shards[i] for i in range(len(test_names))} 

# data is a list of 10 dataset
def get_average_mse(data_set, model, window_size, num_datasets):
    MSE = 0
    data = create_test_data_sets(data_set, num_datasets)
    for test in list(data):
        #forecast (predictions) for every point in the time series
        series_trans = np.array(data[test])
        forecast = []
        for time in range(len(series_trans)-window_size):
            forecast.append(model.predict(series_trans[time:time + window_size][np.newaxis]))
        forecast = forecast[:]
        results = np.array(forecast)[:, 0, 0]
        MSE += tf.keras.metrics.mean_squared_error(series_trans[window_size:len(series_trans)], results).numpy()
    return np.mean(MSE)

In [ ]:
def krum_aggregation(gradients,nbselected,nbworkers, nbbyzwrks):
    """ Aggregate the gradient using the associated (deprecated) native helper.
    Args:
      gradients List of submitted gradients, as numpy arrays
    Returns:
      Aggregated gradient, as a numpy array
    """
    if nbselected == nbworkers:
      # Fast path average
      result = list()
      for i in range(nbworkers):
        result.append(gradients[i])
      return sum_scaled_weights(result)
    else:
      # Compute list of scores
      scores = [list() for i in range(nbworkers)]
      for i in range(nbworkers - 1):
        score = scores[i]
        for j in range(i + 1, nbworkers):
          # With: 0 <= i < j < nbworkers
          distance = squared_distance(weight_to_vector(gradients[i]), weight_to_vector(gradients[j]))
          if math.isnan(distance):
            distance = math.inf
          score.append(distance)
          scores[j].append(distance)
      nbinscore = nbworkers - nbbyzwrks - 2
      for i in range(nbworkers):
        score = scores[i]
        score.sort()
        scores[i] = sum(score[:nbinscore])
      # Return the average of the m gradients with the smallest score
      pairs = [(gradients[i], scores[i]) for i in range(nbworkers)]
      pairs.sort(key=lambda pair: pair[1])
      result= list()
      for i in range(nbselected):
        result.append(pairs[i][0])
      print(len(result))
      return sum_scaled_weights(result)